In [ ]:
%cd /home/maximilianheil/fungiclef-2024/

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pyspark.sql.functions as f
import pyspark.ml as ml
from pyspark.sql.types import ArrayType, FloatType, IntegerType
import fungiclef.embedding.transforms as trans
from fungiclef.utils import get_spark

In [ ]:
spark = get_spark(**{
    "spark.sql.parquet.enableVectorizedReader": False, 
})
display(spark)

In [ ]:
# Get list of stored filed in cloud bucket
root = "gs://dsgt-clef-fungiclef-2024"
! date
! gcloud storage du --readable-sizes {root}/data/parquet/DF20


In [12]:

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/dev/train/"
input_folder = f"dev_train.parquet"

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/"
input_folder = f"DF20"



dev_df = spark.read.parquet(gcs_parquet_path+input_folder)
dev_df.printSchema()
dev_df.count()

In [ ]:
# Get subset of images to test pipeline
dev_subset_df = (
    dev_df
    .limit(100)
    .cache()
)
print(dev_subset_df.count())

In [ ]:
# Init DINOv2 wrapper
dino = trans.WrappedDinoV2(input_col="data", output_col="transformed_data")

# Init Descrite Cosine Transform wrapper
dctn = trans.DCTN(input_col="transformed_data", output_col="dctn_data")

# Create Pipeline
pipeline = ml.Pipeline(stages=[dino, dctn]) #, dctn

# Fit pipeline to DF
model = pipeline.fit(dev_subset_df)

# Apply the model to transform the DF
transformed_df = model.transform(dev_subset_df).cache()

# Show results
transformed_df.select(["ImageUniqueID", "species", "transformed_data"]).show(n=10)